In [1]:
!pip install langchain --quiet
!pip install langchain_community --quiet
!pip install langchain_mistralai --quiet
!pip install huggingface_hub --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from langchain.chains import create_sql_query_chain
from langchain_mistralai import ChatMistralAI
from langchain_community.tools import QuerySQLDataBaseTool
from google.colab import userdata

In [4]:
dt=pd.read_csv('/content/sample_data/california_housing_test.csv',header=0)

engine=create_engine('sqlite:///housing.db')
dt.to_sql('housing',con=engine,if_exists='replace',index=False)

3000

In [5]:
db=SQLDatabase(engine=engine)

In [6]:
db.run("select count(*) from housing;")

'[(3000,)]'

In [7]:
api_key=userdata.get('MISTRAL_AI_API_KEY')
llm=ChatMistralAI(model_name='mistral-large-latest',api_key=api_key)

In [12]:
from langchain_core.prompts import PromptTemplate

create_query_prompt=PromptTemplate(
    input_variables=["input","table_info","top_k"],
    template="""You are an agent designed to interact with a SQL database.
    Your job is to create only the sql query based on the user question.
    Do not produce any other outputs that the correct sql query.
    The query must not contain "\". The query should be clean and executable

    question: {input}
    table_info: {table_info}
    top_k: {top_k}
    query:
    """
)

create_query_chain=create_sql_query_chain(llm,db,create_query_prompt)
execute_query_chain=QuerySQLDataBaseTool(db=db)

In [10]:
create_query_chain.invoke({"question":"how many rows are there in the table?"})

'SELECT COUNT(*) FROM housing;'

In [13]:
chain_1=create_query_chain | execute_query_chain

In [14]:
chain_1.invoke({"question":"how many rows are there in the table?"})

'[(3000,)]'

In [15]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

answer_prompt=PromptTemplate.from_template(
    """
        Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQLQuery: {query}
SQL Result: {result}
Answer: """
)

answer=answer_prompt | llm | StrOutputParser()

chain=(
    RunnablePassthrough.assign(query=create_query_chain).assign(
        result=itemgetter("query") | execute_query_chain
    )|answer
)

In [25]:
chain.invoke({"question":"which house has lowest median_house_vale?"})

'The house with the lowest median house value is located at latitude 32.71 and longitude -117.16, with a median house value of 22500.0.'

In [20]:
db.run("SELECT * FROM housing ORDER BY median_house_value DESC LIMIT 1;")

'[(-118.06, 34.15, 37.0, 1980.0, 226.0, 697.0, 226.0, 15.0001, 500001.0)]'